In [57]:
import torch.nn as nn
import os
import pandas as pd
import numpy as np
import pickle

In [17]:
data_dir = os.path.join(os.getcwd(),"data","ml-100k")

In [18]:
data_dir

'/Users/arnav1712/Desktop/University_Stuff/SWM/RecNet/data/ml-100k'

In [19]:
data_arr = []

In [20]:
with open(os.path.join(data_dir,"u.data")) as f:
    data = f.readlines()

In [21]:
for row in data:
    data_arr.append(row.strip().split("\t"))

In [22]:
data_arr = np.array(data_arr)

In [23]:
data_df = pd.DataFrame(data=data_arr,columns=["user_id","item_id","rating","timestamp"])

In [27]:
data_df = data_df.sort_values(['user_id', 'timestamp'], ascending=[1, 1])

In [30]:
step_size=1
max_len = 10

In [35]:
data_df.head()

,user_id,item_id,rating,timestamp
59972,1,168,5,874965478
92487,1,172,5,874965478
74577,1,165,5,874965518
48214,1,156,4,874965556
15764,1,196,5,874965677


array(['1', '1', '1', ..., '99', '99', '99'], dtype=object)

In [38]:
unique_user_id,indices,counts = np.unique(data_df['user_id'].values,
                                          return_index=True,
                                          return_counts=True)

In [42]:
def sliding_window(item_ids,step_size,max_len):
    
    for i in range(len(item_ids),0,-step_size):
        yield item_ids[max(i-max_len,0):i]

In [88]:

def return_sequences(df,indices,unique_user_id,max_len,step_size):
    
    for idx,start_index in enumerate(indices):
        if idx==len(indices)-1:
            break
        user_id=unique_user_id[idx]

        last_index = indices[idx+1]

        
        for seq in sliding_window(df['item_id'].values[start_index:last_index],step_size,max_len) :
            
            yield (user_id,seq)

In [91]:
seq_arr = []
for seq in return_sequences(data_df,indices,unique_user_id,max_len,step_size):
    seq_arr.append(seq)

In [92]:
!rm sequences.pkl
!touch sequences.pkl

In [93]:
with open("sequences.pkl","wb") as f:
    pickle.dump(seq_arr,f)

In [94]:
arr = pickle.load(open("sequences.pkl","rb"))

('1', array(['209', '32', '189', '242', '171', '111', '256', '5', '74', '102'],
       dtype=object))